In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-25 04:13:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.70MB/s    in 0.2s    

2022-06-25 04:13:23 (5.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



#### DataFrame or table for the vine_table data using one of three methods above

In [8]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

vine_df.show()
vine_df.printSchema()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

#### Data is filtered to create a DataFrame or table where there are 20 or more total votes

In [9]:
over_20_votes_df = vine_df.filter(vine_df['total_votes']>=20)
over_20_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

#### Data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50%

In [14]:
helpful_db = over_20_votes_df.filter((vine_df['helpful_votes'] / vine_df['total_votes']) >= 0.5).withColumn("ratio",(vine_df['helpful_votes'] / vine_df['total_votes']))
helpful_db.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|               1.0|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|0.9655172413793104|
| R6OD85TMEMHQQ|          5|  

#### Data is filtered to create a DataFrame or table where there is a Vine review

In [17]:
#helpful_db.filter("vine='Y'").show()
#or
vine_paid_db = helpful_db.filter(helpful_db['vine']=='Y')
vine_paid_db.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|0.9545454545454546|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|0.8947368421052632|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|0.9782608695652174|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|               1.0|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|0.8979591836734694|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|0.9230769230769231|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|0.9642857142857143|
|R2XRYNV2SY3ZKL|          5|  

#### Data is filtered to create a DataFrame or table where there isn’t a Vine review

In [18]:
#helpful_db.filter(helpful_db['vine']=='N').show()
#or
vine_unpaid_db = helpful_db.filter("vine='N'")
vine_unpaid_db.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|0.9183673469387755|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|               1.0|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|0.5925925925925926|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|0.9047619047619048|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|0.8636363636363636|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|               1.0|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|0.9655172413793104|
| R6OD85TMEMHQQ|          5|  

#### The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews for **Vine** reviews

In [34]:
# Total number of paid reviews
total_number_of_paid_reviews = vine_paid_db.count()
total_number_of_paid_reviews

22

In [36]:
# Total number of 5star paid reviews
total_number_of_5star_paid_reviews = vine_paid_db.filter(vine_paid_db['star_rating']==5).count()
total_number_of_5star_paid_reviews

13

In [37]:
# Percentage of of 5star paid reviews
(total_number_of_5star_paid_reviews/total_number_of_paid_reviews)*100

59.09090909090909

#### The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews for **non-Vine** reviews

In [35]:
# Total number of unpaid reviews
total_number_of_unpaid_reviews = vine_unpaid_db.count()
total_number_of_unpaid_reviews

26987

In [38]:
# Total number of 5star unpaid reviews
total_number_of_5star_unpaid_reviews = vine_unpaid_db.filter(vine_unpaid_db['star_rating']==5).count()
total_number_of_5star_unpaid_reviews

14475

In [33]:
# Percentage of of 5star unpaid reviews
(total_number_of_5star_unpaid_reviews/total_number_of_unpaid_reviews)*100

53.63693630266425